# Assignment \#3: Importing Data (Part 2)
In this Assignment you will be practising alternative ways of sourcing data.

## Web scraping
For this Assignment you are tasked with scraping the following website: 'premierleague.com'

On this website you are looking for the results of football matches that took place in August 2019.

The results of these matches can be found by going to the url 'premierleague.com/match/\[match_ID\]' where match_ID will start at 46605 up to 46644. 

## Examing the code below.
The cell below contains limited capability of scraping the website for the results of a match and requires further functionality.

The code retrieves the html page containing the match results.
Finish the code to retrieve the following <div\> in which both team names as well as the score line for a match can be found:

Find and print the contents of <div class="teamsContainer"\></div\>


In [52]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

url = 'https://premierleague.com/match/46605'

page = requests.get(url)

soup = BeautifulSoup(page.content, 'html.parser')

# insert code here
teams_container = soup.find("div", {"class": "teamsContainer"})
print(teams_container)

<div aria-atomic="true" aria-live="polite" class="teamsContainer">
<div class="team home">
<a aria-hidden="true" class="badgeContainer" href="/clubs/10/Liverpool/overview">
<span class="badge badge--large badge-image-container" data-size="50" data-widget="club-badge-image">
<img class="badge-image badge-image--50" src="https://resources.premierleague.com/premierleague/badges/50/t14.png" srcset="https://resources.premierleague.com/premierleague/badges/50/t14.png, https://resources.premierleague.com/premierleague/badges/50/t14@x2.png 2x">
</img></span>
<span class="badge badge--small badge-image-container" data-size="25" data-widget="club-badge-image">
<img class="badge-image badge-image--25" src="https://resources.premierleague.com/premierleague/badges/25/t14.png" srcset="https://resources.premierleague.com/premierleague/badges/25/t14.png, https://resources.premierleague.com/premierleague/badges/25/t14@x2.png 2x">
</img></span>
</a>
<a aria-hidden="true" class="teamName" href="/clubs/10

In [18]:
type(teams_container)

bs4.element.Tag

## Write code to clean the html code stored in 'teamsContainer' to output the team names and scoreline of the given match.

In [82]:
result = []

In [83]:

# write code here
score = teams_container.find('div',class_="score fullTime")
scoreline = (score.text)
home_score = scoreline[0]
away_score = scoreline[2]

home = soup.find("div", {"class": "team home"}).find('span',class_="long")
result.append(home.text)
result.append(home_score)


away = soup.find("div", {"class": "team away"}).find('span',class_="long")
result.append(away.text)
result.append(away_score)

In [84]:
print(result)

['Liverpool', '4', 'Norwich City', '1']


## Now we want to retrieve the team names as well of the scoreline for all the matches in August (match ID: 46605 to 46643).
Save the names of the home team, away team, the number of goals scored by the home team and the number of goals scored by the away team in a dataframe.

Afterwards print out the dataframe.

Note: If correctly implemented, it should take up to a 3 minutes to scrape these results.

In [87]:
home_teams = []
away_teams = []
home_scores = []
away_scores =[]

In [90]:
# write code here
url = 'https://premierleague.com/match/{0}'

# use list coprehension to get all the urls
all_urls = [url.format(match) for match in range(46605, 46644)]

for url in all_urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    home_team = soup.find("div", {"class": "team home"}).find('span',class_="long")
    away_team = soup.find("div", {"class": "team away"}).find('span',class_="long")
    
    soccer_scores = soup.find('div',class_="score fullTime")
    scoreline = (soccer_scores.text)
    home_score = scoreline[0]
    away_score = scoreline[2]
    
    home_teams.append(home_team.text)
    home_scores.append(home_score)
    away_teams.append(away_team.text)
    away_scores.append(away_score)

In [137]:
import pandas as pd
data_tuples = list(zip(home_teams, home_scores, away_teams, away_scores))
df = pd.DataFrame(data_tuples, columns = ['Home Team', 'Home_Score', 'Away Team', 'Away_Score'])
print(df)

                   Home Team Home_Score                 Away Team Away_Score
0                  Liverpool          4              Norwich City          1
1            AFC Bournemouth          1          Sheffield United          1
2                    Burnley          3               Southampton          0
3                  Liverpool          4              Norwich City          1
4            AFC Bournemouth          1          Sheffield United          1
5                    Burnley          3               Southampton          0
6             Crystal Palace          0                   Everton          0
7             Leicester City          0   Wolverhampton Wanderers          0
8          Tottenham Hotspur          3               Aston Villa          1
9                    Watford          0  Brighton and Hove Albion          3
10           West Ham United          0           Manchester City          5
11         Manchester United          4                   Chelsea          0

## Print a list of all unique teams.

In [188]:
# write code here
all_teams = home_teams + away_teams
unique_teams = set(all_teams)
print(f"There are " + str(len(unique_teams)) + " unique teams pulled during the scraping")
print(f"They are " + str((unique_teams)) )

There are 20 unique teams pulled during the scraping
They are {'Sheffield United', 'Liverpool', 'West Ham United', 'Newcastle United', 'Tottenham Hotspur', 'Aston Villa', 'Manchester United', 'Watford', 'Manchester City', 'Wolverhampton Wanderers', 'Burnley', 'Chelsea', 'Arsenal', 'Everton', 'Brighton and Hove Albion', 'AFC Bournemouth', 'Norwich City', 'Leicester City', 'Crystal Palace', 'Southampton'}


## Using the scores of each match, compute the current standings.
Use the following rules to calculate the points for a given team:

For a win, award a team 3, points.

A draw earns both teams 1 point.

No points are earned for a loss.

### Print the resulting dataframe ordered by total points.
Your dataframe should have the following columns: Team Name, Total points.


In [138]:
#Convert the values from str to int so can do calculations based on the scoreline
df = df.astype({"Home_Score":"int","Away_Score":"int"}) 

In [139]:
#Find Home Score run
def categorise_home(row):  
    if row['Home_Score'] > row['Away_Score']:
        return 3
    elif row['Home_Score'] == row['Away_Score']:
        return 1
    else:
        return 0

In [140]:
df['Home_Points'] = df.apply(lambda row: categorise_home(row), axis=1) #add a new column

In [141]:
#Find Home Score
def categorise_away(row):  
    if row['Home_Score'] < row['Away_Score']:
        return 3
    elif row['Home_Score'] == row['Away_Score']:
        return 1
    else:
        return 0


In [189]:
df['Away_Points'] = df.apply(lambda row: categorise_away(row), axis=1) #add new column

In [190]:
# write code here
print(df)

                   Home Team  Home_Score                 Away Team  \
0                  Liverpool           4              Norwich City   
1            AFC Bournemouth           1          Sheffield United   
2                    Burnley           3               Southampton   
3                  Liverpool           4              Norwich City   
4            AFC Bournemouth           1          Sheffield United   
5                    Burnley           3               Southampton   
6             Crystal Palace           0                   Everton   
7             Leicester City           0   Wolverhampton Wanderers   
8          Tottenham Hotspur           3               Aston Villa   
9                    Watford           0  Brighton and Hove Albion   
10           West Ham United           0           Manchester City   
11         Manchester United           4                   Chelsea   
12          Newcastle United           0                   Arsenal   
13                  

In [191]:
home_new = df[['Home Team', 'Home_Points']]
away_new = df[['Away Team', 'Away_Points']]

In [192]:
home_new.rename(columns = {'Home Team':'team_name', 'Home_Points':'points'}, inplace = True)
away_new.rename(columns = {'Away Team':'team_name', 'Away_Points':'points'}, inplace = True)


/opt/anaconda3/lib/python3.9/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [193]:
df3 = pd.concat([home_new,away_new], ignore_index=True)

In [194]:
final_table = df3.groupby(['team_name'])['points'].sum()

In [195]:
print(final_table)

team_name
AFC Bournemouth              5
Arsenal                      7
Aston Villa                  3
Brighton and Hove Albion     4
Burnley                      7
Chelsea                      5
Crystal Palace               7
Everton                      7
Leicester City               8
Liverpool                   15
Manchester City             10
Manchester United            5
Newcastle United             4
Norwich City                 3
Sheffield United             6
Southampton                  4
Tottenham Hotspur            5
Watford                      1
West Ham United              4
Wolverhampton Wanderers      3
Name: points, dtype: int64


In [196]:
table = final_table.to_frame()

In [197]:
table.sort_values("points", ascending=False)

,points
team_name,
Liverpool,15
Manchester City,10
Leicester City,8
Arsenal,7
Burnley,7
Crystal Palace,7
Everton,7
Sheffield United,6
Tottenham Hotspur,5
